In [21]:
# Dependencies.

import pandas as pd
import numpy as np

# Language detection and translation libraries.

from deep_translator import GoogleTranslator

# Language Processing
import nltk
from nltk.tokenize import word_tokenize # Tokenizing the message
from nltk.stem import WordNetLemmatizer # Lemmatization of the message

To-Do:

1. Aniket pointed out that a lot of the words are in latin. `deep_translator` is a library for language translation. `GoogleTranslator` is an object that uses Google Translate's API for translation. We want to convert the latin words into english.

2. A lot of the words used are in english, except with an 'x' or a 'z' at the ending. Removing this suffix would make the word closer to english, while also removing a garbage character that doesn't add any context/meaning to the word.

In [22]:
df = pd.read_csv("data.csv")

df

,message,fingers,tail,species
0,pluvia arbor aquos,4,no,Aquari
1,cosmix xeno nebuz odbitaz,5,yes,Zorblax
2,solarix glixx novum galaxum quasar,5,yes,Zorblax
3,arbor insectus pesros ekos dootix nimbus,2,yes,Florian
4,mermax drakos lorix epikoz deftax,4,no,Faerix
...,...,...,...,...
495,empathix sadix disgux dredax pridius afgstix e...,2,no,Emotivor
496,quasar ustron nebulax meteorn,4,no,Quixnar
497,astron xeno ceaestar astron kometa,6,yes,Zorblax
498,sporzom nimbus terram terranix aviana ekos nimbub,2,yes,Florian


In [29]:
# Removes 'x' and 'z' from the ending.

def cleanEnding(message):

    cleaned_message = []
    message = message.split()
    
    for word in message:
        if word[-1] in ['x','z']:
            word = word[:-1]
        cleaned_message.append(word)
    return ' '.join(cleaned_message)

# Convert latin words to english.

def englishConversion(message):

    translator = GoogleTranslator(source="latin",target="english") # Initialize the translator.

    translatedMessage = []
    message = message.split(' ')

    for word in message:
        translatedWord = translator.translate(word)
        translatedMessage.append(translatedWord)
    
    return ' '.join(translatedMessage)

def processing(message):

    message = englishConversion(cleanEnding(message))

    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(message)

    lemmatizations = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatizations
    


In [30]:
df['preprocessed_message'] = df['message'].apply(processing)

In [50]:
# Cleaning the data further

def join(message):
    return ' '.join(word for word in message)

from nltk.corpus import stopwords

nltk.download('stopwords')
stopwords_list = stopwords.words('english')

def remove_stopwords(message):

    res = []
    message = message.split()

    for word in message:
        if word not in stopwords_list:
            res.append(word)
    
    return ' '.join(res)

df['preprocessed_message_str'] = df['preprocessed_message'].apply(join)
df['preprocessed_message_str'] = df['preprocessed_message_str'].apply(remove_stopwords)

df.rename(columns={'preprocessed_message_str':'processedMessage'},inplace=True)
df.to_csv("processedData.csv")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\suhan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
